# Estimate stimulation short-term effects using time shifts and amplitude differences

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

from IPython.display import HTML

import seppy
import os

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=150, xmax=250, title=None, grid=True, colorbar=False, curve=np.array([]), linestyle='-', animation=False, savefig=None):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    if animation==False:
        fig=plt.figure(figsize=(5, 3),dpi=200)
        
    im = plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offset (m)')
    plt.ylabel('Time (s)')
    if title!=None:
        plt.title(r""+title)
    if grid==True:
        plt.grid(color='r', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    if animation==False:
        if savefig != None:
            plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
        else:
            plt.show()
    else:
        return im

def tshifts(data1, data2, dt):
    """Compute time shifts"""
    ns = data1.shape[0]
    nt = data1.shape[1]
    tshifts = np.zeros((ns))
    for s in range(ns):
        # 1d correlation
        corr=np.correlate(data1[s,:],data2[s,:],mode='full')
        # maximum correlation
        ind_max=np.argmax(corr)
        tshifts[s]=ind_max-(nt-1)
    return dt*tshifts

def applyShifts(data, tshifts, dt):
    """Apply time shifts for QC"""
    data2=np.copy(data)
    ns = data.shape[0]
    ntr = data.shape[1]
    for s in range(ns):
        for itr in range(ntr):
            data2[s,itr,:]=ndimage.shift(data[s,itr,:],tshifts[s]/dt)
    return data2

In [ ]:
# read data and synthetic, group2, perf>=18 

sep = seppy.sep()

# field data unstimulated side
data_axes, data = sep.read_file(datapath+"ch5un_data_st_group2.H.bp30100")
pdata = data.reshape(data_axes.n,order='F').T
pdata = pdata[:,:,:101]
data_un = np.flip(pdata, axis=1)

# field data stimulated side
data_axes, data = sep.read_file(datapath+"ch5st_data_group2.H.bp30100")
pdata = data.reshape(data_axes.n,order='F').T
data_st = pdata[:,:101,:101]

# matched synthetic unstimulated side
data_axes, data = sep.read_file(datapath+"ch5un_syn_group2_matched30100.H.bp30100")
pdata = data.reshape(data_axes.n,order='F').T
pdata = pdata[:,:,:101]
syn_un = np.flip(pdata, axis=1)

# matched synthetic stimulated side
data_axes, data = sep.read_file(datapath+"ch5st_syn3_group2_matched30100.H.bp30100")
pdata = data.reshape(data_axes.n,order='F').T
syn_st = pdata[:,:101,:101]

nt=syn_st.shape[2]
ntr=syn_st.shape[1]
ns=data_axes.n[2]
dt=data_axes.d[0]
dx=data_axes.d[1]

stages=np.array([6,7,10,11,13,14,15,16,18,19,20,21,23,24,27,28]).astype('int')

In [ ]:
sp=14
# plotseis(data_un[sp,:,:], valmin=-4, valmax=4)
# plotseis(syn_un[sp,:,:] , valmin=-4, valmax=4)
plotseis(syn_st[sp,:,:] , valmin=-2, valmax=2)
plotseis(data_st[sp,:,:], valmin=-2, valmax=2)

In [ ]:
# compute P- and S-wave shifts for the unstimulated part
tshifts_un=np.zeros((ns,2))
itr0_p=0
it0_p=20
nt0_p=11
itr0_s=100
it0_s=73
nt0_s=11

tshifts_un[:,0]=tshifts(data_un[:,itr0_p,it0_p:it0_p+nt0_p], syn_un[:,itr0_p,it0_p:it0_p+nt0_p], dt)
tshifts_un[:,1]=tshifts(data_un[:,itr0_s,it0_s:it0_s+nt0_s], syn_un[:,itr0_s,it0_s:it0_s+nt0_s], dt)

# compute P- and S-wave shifts for the stimulated part
tshifts_st=np.zeros((ns,2))
itr0_p=0
it0_p=20
nt0_p=11
itr0_s=0
it0_s=50
nt0_s=13

tshifts_st[:,0]=tshifts(data_st[:,itr0_p,it0_p:it0_p+nt0_p], syn_st[:,itr0_p,it0_p:it0_p+nt0_p], dt)
tshifts_st[:,1]=tshifts(data_st[:,itr0_s,it0_s:it0_s+nt0_s], syn_st[:,itr0_s,it0_s:it0_s+nt0_s], dt)

In [ ]:
fig=plt.figure(figsize=(6.66, 2),dpi=300)

plt.subplot(1,2,1)
plt.scatter(stages,1000*(tshifts_st[:,0]-tshifts_un[:,0]),s=50,marker='o',label='P wave')
plt.scatter(stages,1000*(tshifts_st[:,1]-tshifts_un[:,1]),s=40,marker='*',label='SV wave')
plt.xlabel('Stage number')
plt.ylabel('Time shift (ms)')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r"a)",loc='left')
plt.legend()

plt.subplot(1,2,2)
bins=1000*np.linspace(-0.002,0.008,6)-1
plt.hist(1000*(tshifts_st[:,0]-tshifts_un[:,0]),bins=bins,alpha=1.0,edgecolor='k',label='P wave')
plt.hist(1000*(tshifts_st[:,1]-tshifts_un[:,1]),bins=bins,alpha=0.5,edgecolor='k',label='SV wave')
plt.xlabel('Time shift (ms)')
plt.ylabel('Stage count')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r"b)",loc='left')
plt.legend()

plt.tight_layout()
# plt.show()

plt.savefig(figpath+'ch5st_tshifts_group2.png',bbox_inches='tight',format='png')

p_shift_med=np.median(1000*(tshifts_st[:,0]-tshifts_un[:,0]))
s_shift_med=np.median(1000*(tshifts_st[:,1]-tshifts_un[:,1]))

print("Median P shift = %f ms" %p_shift_med)
print("Median S shift = %f ms" %s_shift_med)

The relation to use is $\delta V = -V^2 \frac{\delta t}{d}$ where $\delta t$ is time shift, $d$ is the near offset (150 m), and $V$ is the background horizontal velocity.

In [ ]:
vp90=4250
vs90=1700
offmin=150
print("VP perturbation = %.0f m/s or %.0f%%" %(-0.001*vp90**2*p_shift_med/offmin,-0.001*100*vp90*p_shift_med/offmin))
print("VS perturbation = %.0f m/s or %.0f%%" %(-0.001*vs90**2*s_shift_med/offmin,-0.001*100*vs90*s_shift_med/offmin))

In [ ]:
# read frequency-velocity amplitude spectra for data
# |offset| = [150 - 650 m] or [150 - 200 m] or [200 - 650 m]
# unstimulated part, negative offsets, perf <=80, stages [2,3,4,5,6,7,10,11,13,14,15,16,18,19,20,21]
# stimulated part, positive offsets, perf>=18, stages [6,7,10,11,13,14,15,16,18,19,20,21,23,24,27,28]

sep = seppy.sep()

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.H.fv")
pdata = data.reshape(data_axes.n,order='F').T
pdata = pdata[:,:,10:]
fv_un = np.flip(pdata, axis=1)

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.H.fv.near")
pdata = data.reshape(data_axes.n,order='F').T
pdata = pdata[:,:,10:]
fv_un_near = np.flip(pdata, axis=1)

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.H.fv.far")
pdata = data.reshape(data_axes.n,order='F').T
pdata = pdata[:,:,10:]
fv_un_far = np.flip(pdata, axis=1)

data_axes, data = sep.read_file(datapath+"ch5st_data_group2.H.fv")
pdata = data.reshape(data_axes.n,order='F').T
fv_st = pdata[:,:,10:]

data_axes, data = sep.read_file(datapath+"ch5st_data_group2.H.fv.near")
pdata = data.reshape(data_axes.n,order='F').T
fv_st_near = pdata[:,:,10:]

data_axes, data = sep.read_file(datapath+"ch5st_data_group2.H.fv.far")
pdata = data.reshape(data_axes.n,order='F').T
fv_st_far = pdata[:,:,10:]

df=data_axes.d[0]
dv=0.001*data_axes.d[1]
of=data_axes.o[0]+10*df
ov=0.001*data_axes.o[1]
nf=fv_st.shape[2]
nv=fv_st.shape[1]

In [ ]:
# compute average spectra over central stages [6,7,10,11,13,14,15,16,18,19,20,21]
stages_central=np.array([6,7,10,11,13,14,15,16,18,19,20,21]).astype('int')
stages_central_int = np.arange(4,12,1).astype('int')
fv_un_avg = np.average(fv_un[stages_central_int,:,:], axis=0)
fv_un_near_avg = np.average(fv_un_near[stages_central_int,:,:], axis=0)
fv_un_far_avg = np.average(fv_un_far[stages_central_int,:,:], axis=0)
fv_st_avg = np.average(fv_st[stages_central_int,:,:], axis=0)
fv_st_near_avg = np.average(fv_st_near[stages_central_int,:,:], axis=0)
fv_st_far_avg = np.average(fv_st_far[stages_central_int,:,:], axis=0)

In [ ]:
# Pick the maxima in a given window containing P-event only
fmin=125
vmin=4

ifmin=int((fmin-of)/df)
ivmin=int((vmin-ov)/dv)

d = fv_un_near_avg[ivmin:,ifmin:]
a = np.asarray(np.unravel_index(np.argmax(d), d.shape)) + [ivmin, ifmin]
fv_un_near_avg_max=np.array([a[0]*dv+ov,a[1]*df+of,fv_un_near_avg[a[0],a[1]]])

d = fv_un_far_avg[ivmin:,ifmin:]
a = np.asarray(np.unravel_index(np.argmax(d), d.shape)) + [ivmin, ifmin]
fv_un_far_avg_max=np.array([a[0]*dv+ov,a[1]*df+of,fv_un_far_avg[a[0],a[1]]])

d = fv_st_near_avg[ivmin:,ifmin:]
a = np.asarray(np.unravel_index(np.argmax(d), d.shape)) + [ivmin, ifmin]
fv_st_near_avg_max=np.array([a[0]*dv+ov,a[1]*df+of,fv_st_near_avg[a[0],a[1]]])

d = fv_st_far_avg[ivmin:,ifmin:]
a = np.asarray(np.unravel_index(np.argmax(d), d.shape)) + [ivmin, ifmin]
fv_st_far_avg_max=np.array([a[0]*dv+ov,a[1]*df+of,fv_st_far_avg[a[0],a[1]]])

In [ ]:
sp=15
data = fv_un_far_avg
vmin=0
vmax=4000

fig=plt.figure(figsize=(6.66, 6),dpi=300)

plt.subplot(2,2,1)
plt.imshow(fv_un_near_avg, aspect='auto',interpolation='spline36',extent=[of, of+(nf-1)*df, ov+(nv-1)*dv, ov],vmin=vmin, vmax=vmax/5,cmap='jet')
plt.scatter(fv_un_near_avg_max[1],fv_un_near_avg_max[0],s=30,color='k',marker='x')
plt.ylabel('Phase velocity (km/s)')
plt.gca().set_xticklabels([])
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r"a)                     Near offsets",loc='left')
plt.grid(color='w', linestyle='-', linewidth=0.2)

plt.subplot(2,2,2)
plt.imshow(fv_un_far_avg, aspect='auto',interpolation='spline36',extent=[of, of+(nf-1)*df, ov+(nv-1)*dv, ov],vmin=vmin, vmax=vmax,cmap='jet')
plt.scatter(fv_un_far_avg_max[1],fv_un_far_avg_max[0],s=30,color='k',marker='x')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r"                       Mid-far offsets",loc='left')
plt.grid(color='w', linestyle='-', linewidth=0.2)

plt.subplot(2,2,3)
plt.imshow(fv_st_near_avg, aspect='auto',interpolation='spline36',extent=[of, of+(nf-1)*df, ov+(nv-1)*dv, ov],vmin=vmin, vmax=vmax/5,cmap='jet')
plt.scatter(fv_st_near_avg_max[1],fv_st_near_avg_max[0],s=30,color='k',marker='x')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase velocity (km/s)')
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r"b)",loc='left')
plt.grid(color='w', linestyle='-', linewidth=0.2)

plt.subplot(2,2,4)
plt.imshow(fv_st_far_avg, aspect='auto',interpolation='spline36',extent=[of, of+(nf-1)*df, ov+(nv-1)*dv, ov],vmin=vmin, vmax=vmax,cmap='jet')
plt.scatter(fv_st_far_avg_max[1],fv_st_far_avg_max[0],s=30,color='k',marker='x')
plt.xlabel('Frequency (Hz)')
plt.gca().set_yticklabels([])
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)

plt.tight_layout()

plt.savefig(figpath+'ch5st_fv_spectra_group2.png',bbox_inches='tight',format='png')

print("Max for near offset unstimulated %f at %f Hz" %(fv_un_near_avg_max[2],fv_un_near_avg_max[1]))
print("Max for near offset   stimulated %f at %f Hz" %(fv_st_near_avg_max[2],fv_st_near_avg_max[1]))
print("Max for mid-far offset unstimulated %f at %f Hz" %(fv_un_far_avg_max[2],fv_un_far_avg_max[1]))
print("Max for mid-far offset   stimulated %f at %f Hz" %(fv_st_far_avg_max[2],fv_st_far_avg_max[1]))
print("Near st/un ratio %f" %(fv_st_near_avg_max[2]/fv_un_near_avg_max[2]))
print("Mid-far st/un ratio %f" %(fv_st_far_avg_max[2]/fv_un_far_avg_max[2]))

#### Estimate $\Delta Q$ for P wave
The amplitude function can be written as (after omitting the geometrical spreading) $$A(x)=A(x_0)\exp(\frac{- \pi f x}{c Q}).$$
I deduce an estimate of $\Delta(1/Q)=(1/Q_{st}-1/Q_{un})$:
$$\Delta(1/Q)=\frac{c}{\pi f x}\log(A_{st}/A_{un}).$$
I set $x=175$ m or $x=425$ m

In [ ]:
xnear=175
xfar=425
f = 160
c = 4300.
Q0 = 60
dQnear = c/(math.pi*f*xnear)*math.log(fv_un_near_avg_max[2]/fv_st_near_avg_max[2])
dQfar = c/(math.pi*f*xfar)*math.log(fv_un_far_avg_max[2]/fv_st_far_avg_max[2])
print("Delta 1/Q for near offsets %.3f or Delta Q = %.0f assuming a reference Qp of %.0f" %(dQnear,-dQnear*Q0**2/(1+Q0*dQnear),Q0))
print("Delta 1/Q for far offsets %.3f or Delta Q = %.0f assuming a reference Qp of %.0f" %(dQfar,-dQfar*Q0**2/(1+Q0*dQfar),Q0))